In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler
from matplotlib.lines import Line2D
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import random

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pandas import json_normalize
from random import randint
from time import sleep

In [3]:
popv = pd.read_csv('popvort.csv')
popv.head()

,artists,songs
0,Shaboozey,A Bar Song (Tipsy)
1,Kendrick Lamar,Not Like Us
2,Teddy Swims,Lose Control
3,"Falling In Reverse, Tech N9ne & Alex Terrible",Ronald
4,Kendrick Lamar,euphoria


In [4]:
#list = pd.read_csv('cluster_list_dropped_d.csv')
#list

In [5]:
list = pd.read_csv('music.csv')
list

,song_name,name,song_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Die in This Town,The Seige,0bqrFwY1HixfnusFxhYbDl,56.0,0.685,0.746,5.0,-4.558,1.0,0.2740,...,0.320,0.594,64.950,audio_features,0bqrFwY1HixfnusFxhYbDl,spotify:track:0bqrFwY1HixfnusFxhYbDl,https://api.spotify.com/v1/tracks/0bqrFwY1Hixf...,https://api.spotify.com/v1/audio-analysis/0bqr...,151500.0,4.0
1,Grand,Kane Brown,4WZ3liyYz5ZzDcL4ibAuOi,49.0,0.583,0.856,6.0,-5.446,0.0,0.0445,...,0.224,0.480,168.053,audio_features,4WZ3liyYz5ZzDcL4ibAuOi,spotify:track:4WZ3liyYz5ZzDcL4ibAuOi,https://api.spotify.com/v1/tracks/4WZ3liyYz5Zz...,https://api.spotify.com/v1/audio-analysis/4WZ3...,178973.0,4.0
2,Grand,Kane Brown,4WZ3liyYz5ZzDcL4ibAuOi,49.0,0.583,0.856,6.0,-5.446,0.0,0.0445,...,0.224,0.480,168.053,audio_features,4WZ3liyYz5ZzDcL4ibAuOi,spotify:track:4WZ3liyYz5ZzDcL4ibAuOi,https://api.spotify.com/v1/tracks/4WZ3liyYz5Zz...,https://api.spotify.com/v1/audio-analysis/4WZ3...,178973.0,4.0
3,Statement,NEFFEX,39kS4TwUw5yTJETWusq2VS,54.0,0.827,0.791,1.0,-6.030,1.0,0.0841,...,0.166,0.720,95.000,audio_features,39kS4TwUw5yTJETWusq2VS,spotify:track:39kS4TwUw5yTJETWusq2VS,https://api.spotify.com/v1/tracks/39kS4TwUw5yT...,https://api.spotify.com/v1/audio-analysis/39kS...,186947.0,4.0
4,Statement,NEFFEX,39kS4TwUw5yTJETWusq2VS,54.0,0.827,0.791,1.0,-6.030,1.0,0.0841,...,0.166,0.720,95.000,audio_features,39kS4TwUw5yTJETWusq2VS,spotify:track:39kS4TwUw5yTJETWusq2VS,https://api.spotify.com/v1/tracks/39kS4TwUw5yT...,https://api.spotify.com/v1/audio-analysis/39kS...,186947.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14217,Crossed The Line,Linn Sandin,1j3N4TCykSmXy12iUiMQWk,57.0,0.533,0.723,2.0,-7.076,0.0,0.1990,...,0.297,0.388,176.096,audio_features,1j3N4TCykSmXy12iUiMQWk,spotify:track:1j3N4TCykSmXy12iUiMQWk,https://api.spotify.com/v1/tracks/1j3N4TCykSmX...,https://api.spotify.com/v1/audio-analysis/1j3N...,147273.0,4.0
14218,Crossed The Line,Vosai,1j3N4TCykSmXy12iUiMQWk,57.0,0.533,0.723,2.0,-7.076,0.0,0.1990,...,0.297,0.388,176.096,audio_features,1j3N4TCykSmXy12iUiMQWk,spotify:track:1j3N4TCykSmXy12iUiMQWk,https://api.spotify.com/v1/tracks/1j3N4TCykSmX...,https://api.spotify.com/v1/audio-analysis/1j3N...,147273.0,4.0
14219,Crossed The Line,Facading,1j3N4TCykSmXy12iUiMQWk,57.0,0.533,0.723,2.0,-7.076,0.0,0.1990,...,0.297,0.388,176.096,audio_features,1j3N4TCykSmXy12iUiMQWk,spotify:track:1j3N4TCykSmXy12iUiMQWk,https://api.spotify.com/v1/tracks/1j3N4TCykSmX...,https://api.spotify.com/v1/audio-analysis/1j3N...,147273.0,4.0
14220,Crossed The Line,Linn Sandin,1j3N4TCykSmXy12iUiMQWk,57.0,0.533,0.723,2.0,-7.076,0.0,0.1990,...,0.297,0.388,176.096,audio_features,1j3N4TCykSmXy12iUiMQWk,spotify:track:1j3N4TCykSmXy12iUiMQWk,https://api.spotify.com/v1/tracks/1j3N4TCykSmX...,https://api.spotify.com/v1/audio-analysis/1j3N...,147273.0,4.0


In [6]:
feature_columns = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
feature_vectors = list[feature_columns].values
kmeans = KMeans(n_clusters=20, random_state=1234, n_init=10)
cluster_numbers = kmeans.fit_predict(feature_vectors)
list['cluster'] = cluster_numbers
list = list[['song_name', 'name', 'song_id', 'cluster']]

In [7]:
list.drop_duplicates(inplace=True)
list.reset_index(drop=True, inplace=True)

C:\Users\lilly\AppData\Local\Temp\ipykernel_22084\588206073.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  list.drop_duplicates(inplace=True)


In [8]:
list

,song_name,name,song_id,cluster
0,Die in This Town,The Seige,0bqrFwY1HixfnusFxhYbDl,11
1,Grand,Kane Brown,4WZ3liyYz5ZzDcL4ibAuOi,16
2,Statement,NEFFEX,39kS4TwUw5yTJETWusq2VS,5
3,Wanchu Back,Aero Chord,3JIkZgwtTlbna1WEU9Qd5b,6
4,abcdefu,GAYLE,4fouWK6XVHhzl78KzQ1UjL,6
...,...,...,...,...
2562,Beanos Theme Song,Holy Tony,2VinaMWAm1thWcb1EHOHso,1
2563,Crossed The Line,Vosai,1j3N4TCykSmXy12iUiMQWk,3
2564,Crossed The Line,Facading,1j3N4TCykSmXy12iUiMQWk,3
2565,Crossed The Line,Linn Sandin,1j3N4TCykSmXy12iUiMQWk,3


In [9]:
#N1 prototype for input
#song = input('Enter a song that you like: ')
#if song in popv.songs.to_list():
#    print('You might also like: ', random.choice(popv.songs))
#else:
#    print('Try a different song.')

In [10]:
secrets_file = open(".gitignore/secrets.txt","r")
string = secrets_file.read()
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

#The code under is done with help of chatgpt and extra personal/lab information

In [12]:
# Define the function to get audio features
def get_audio_features(track_ids):
    return np.array([[t['danceability'], t['energy'], t['loudness'], t['speechiness'], t['acousticness'], t['instrumentalness'], t['liveness'], t['valence'], t['tempo']] for t in sp.audio_features(track_ids)])

# Define the function to search for a song and retrieve its features
def search_song_and_cluster(song_name):
    results = sp.search(q='track:' + song_name, type='track', limit=1)
    if results['tracks']['items']:
        track_info = results['tracks']['items'][0]
        track_id, track_name, artist_name = track_info['id'], track_info['name'], track_info['artists'][0]['name']
        audio_features = get_audio_features([track_id])
        return track_name, artist_name, audio_features
    else:
        return None

# Define the main function
def main():
    song_name = input("Enter the name of the song: ")
    result = search_song_and_cluster(song_name)
    cluster_number = None  # Initialize cluster number variable
    if song_name in popv.songs.to_list():
        print('You might also like: ', random.choice(popv.songs))
    if result:
        track_name, artist_name, feature_vector = result
        additional_songs = sp.search(q='track:', type='track', limit=20)['tracks']['items']
        additional_audio_features = get_audio_features([song['id'] for song in additional_songs])
        all_audio_features = np.vstack((feature_vector, additional_audio_features))

        
        cluster_number = KMeans(n_clusters=20, random_state=1234, n_init=10).fit(all_audio_features).predict(feature_vector.reshape(1, -1))[0]
        print(f"Song '{track_name}' by {artist_name} belongs to cluster number {cluster_number}.")
    else:
        print("Song not found on Spotify.")
    
    return cluster_number  # Return cluster number

if __name__ == "__main__":
    cluster_number = main()



result = list[list['cluster'] == cluster_number].head(3)
display("You may like it:", result)
#I still have the feeling that I did something wrong, especially with comparing it with my clusters on dataframe, but it works. 
#especially happy that I can type it also not fully correctly and it will try to search the closest  searching result

Enter the name of the song:  trolo


Song 'Trololo' by Russian Music Band belongs to cluster number 3.


'You may like it:'

,song_name,name,song_id,cluster
9,Blinding Lights,The Weeknd,0VjIjW4GlUZAMYd2vXMi3b,3
21,Howling,Cartoon,1FUbkX4ySdsOHoUOkyOJte,3
22,Howling,Asena,1FUbkX4ySdsOHoUOkyOJte,3
